# 주정차 단속 현황의 주소를 기반으로 위,경도 데이터 추출

In [181]:
from concurrent.futures import ThreadPoolExecutor
import geocoder

In [182]:
import pandas as pd
import numpy as np

In [183]:
df = pd.read_csv('인프라\\서울특별시 광진구_주정차단속현황_20230331.csv',encoding='cp949')

In [184]:
df.shape[0]

503300

- 총 추출해야 하는 데이터 50만개
- 그러나 대부분의 데이터에 정확한 주소가 기재되지 않음.
- 모든 데이터에 대한 위경도를 추출할 수는 없으나, 각 행에 대한 주소를 적용으로써 최대한 많은 위,경도를 추출하고자 함

In [185]:
arr = [i for i in range(0,df.shape[0])]
# 반복할 전체 행의 인덱스 리스트
df = df.iloc[arr]
df['단속일시'] = df['단속일시'].str.slice(stop=10)
df['단속일시'] = pd.to_datetime(df['단속일시'])
# 단속일시 속성변경
df.reset_index(inplace=True,drop=True)


In [186]:
df

,단속일시,과태료,단속지역,단속장소,위반내용,견인지시,단속구분
0,2015-01-01,32000,화양동,119-4,주정차금지,견인처리,PDA
1,2015-01-01,40000,화양동,건대입구1번출구앞,32조내지 34조위반,미견인,CCTV
2,2015-01-01,40000,구의동,동서울터미널 주차장앞,32조내지 34조위반,미견인,CCTV
3,2015-01-01,40000,화양동,건대입구1번출구앞,32조내지 34조위반,미견인,CCTV
4,2015-01-01,32000,화양동,건대입구1번출구앞,32조내지 34조위반,미견인,CCTV
...,...,...,...,...,...,...,...
503295,2023-03-31,32000,중곡동,서울 광진구 중곡동 192-30,보도,미견인,스마트앱(서울시)
503296,2023-03-31,64000,광장동,320-15,특별구역,미견인,안전신문고(행안부)
503297,2023-03-31,32000,중곡동,서울 광진구 중곡동 669,보도,미견인,PDA
503298,2023-03-31,32000,중곡동,서울 광진구 중곡동 669,보도,미견인,PDA


In [187]:
arr= []
for i in range(df.shape[0]):
    try :
        a = df.loc[i,'단속장소'][-1:]
        a = int(a)
    except :
        continue
    arr.append(i)
    
# 위경도 데이터를 추출하기 위한 지번 주소로 이뤄진 행 추출

In [188]:
num_place = df.iloc[arr]['단속장소']
# 지번 주소 데이터 프레임 성

In [189]:
num_place

0                     119-4
15                 능동로32길81
53                     1-20
86                    374-7
87                   361-26
                ...        
503295    서울 광진구 중곡동 192-30
503296               320-15
503297       서울 광진구 중곡동 669
503298       서울 광진구 중곡동 669
503299    서울 광진구 군자동 473-17
Name: 단속장소, Length: 206589, dtype: object

In [190]:
df = df[df['단속장소'].str.slice(start=-1).isin([str(i) for i in range(10)])]
df['단속장소'].str.slice(stop=1)
# 지번 주소의 앞자리가 서울시나 광진구가 아닐 경우 추출 

0         1
15        능
53        1
86        3
87        3
         ..
503295    서
503296    3
503297    서
503298    서
503299    서
Name: 단속장소, Length: 206589, dtype: object

In [191]:
df.loc[df['단속장소'].str.slice(stop=1).isin([str(i) for i in range(10)]),['단속장소']] = '서울 광진구 ' + df['단속지역'] + ' ' + df['단속장소']
# 지번 주소로 이뤄진 데이터에 대해 서울시 광진구 단속동 정보 기재

In [192]:
df.loc[df['단속장소'].str.slice(stop=1)!='서',['단속장소']] = '서울 광진구 ' + df['단속지역'] + ' ' + df['단속장소']
# 지번 주소로 이뤄지지 않고, 광진구로 시작하거나 단속동 등으로 시작하는 데이터에 대해 서울 광진구 정보 추가

In [193]:
df.reset_index(drop=True,inplace=True)
# 인덱스를 기준으로 반복하기 위해 인덱스 초기화

In [194]:
df

,단속일시,과태료,단속지역,단속장소,위반내용,견인지시,단속구분
0,2015-01-01,32000,화양동,서울 광진구 화양동 119-4,주정차금지,견인처리,PDA
1,2015-01-01,40000,능동,서울 광진구 능동 능동로32길81,주정차금지,견인처리,PDA
2,2015-01-01,40000,자양동,서울 광진구 자양동 1-20,주정차금지,견인처리,PDA
3,2015-01-02,32000,군자동,서울 광진구 군자동 374-7,주정차위반,미견인,주행형CCTV
4,2015-01-02,32000,군자동,서울 광진구 군자동 361-26,주정차위반,미견인,주행형CCTV
...,...,...,...,...,...,...,...
206584,2023-03-31,32000,중곡동,서울 광진구 중곡동 192-30,보도,미견인,스마트앱(서울시)
206585,2023-03-31,64000,광장동,서울 광진구 광장동 320-15,특별구역,미견인,안전신문고(행안부)
206586,2023-03-31,32000,중곡동,서울 광진구 중곡동 669,보도,미견인,PDA
206587,2023-03-31,32000,중곡동,서울 광진구 중곡동 669,보도,미견인,PDA


In [195]:
df['단속구분'].value_counts()
# 약 20만개의 열을 가진 데이터 프레임 생성
# 캐싱을 통해 저장된 데이터는 로컬에서 빠르게 로드할 수 있도록 조치
# 만약 캐싱할 수 없는  데이터에는 지오코딩 api를 활용할 수 있도록 함
df = df[df['단속일시']>='2020-03-17']

### 지오코딩을 통한 주소의 위경도화
1. 약 5만개의 데이터를 지오코딩을 통해 변환하기 위해 소요되는 시간이 지나치게 많음.
2. 따라서 dict 형태에 이미 저장된 매핑 정보가 있다면 우선적으로 확인하고 없다면 지오코딩을 수행한 다음 dict에 저장하도록 코딩

In [196]:
addresses = pd.array(df['단속장소'])

In [197]:
#geo_dict ={}
# 사용 조심..

In [198]:
import pickle
with open('geo_dict.pickle', 'rb') as fr:
    geo_dict = pickle.load(fr)
# 중간에 저장한 dict 불러오기

In [199]:
def extract_geo(i):
    if i in geo_dict.keys():
        geo_list = geo_dict[i]
        # dict에 주소 지오코딩 값이 있다면 참조
    else :
        geo_value = geocoder.osm(i)
        geo_dict[i] = [geo_value.lat,geo_value.lng]
        geo_list = [geo_value.lat,geo_value.lng]
        # 없다면 딕셔너리 키로 주소값을, value로 위,경도 값을 삽입
    print(i)
    return geo_list

In [200]:
for i in addresses:
    extract_geo(i)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [201]:
import pickle
with open('geo_dict.pickle','wb') as fw:
    pickle.dump(geo_dict, fw)
# dict 중간 저장

In [202]:
geo_dict

{'서울 광진구 중곡동 482-1': [37.56286, 127.08921],
 '서울 광진구 구의동 632-3': [37.54417, 127.09262],
 '서울특별시 광진구 긴고랑로7길 76': [37.5640119, 127.0794286],
 '서울 광진구 중곡동 265-17': [37.56286, 127.08921],
 '서울 광진구 구의동 662-1': [37.54417, 127.09262],
 '서울 광진구 중곡동 128-13': [37.56286, 127.08921],
 '서울 광진구 중곡동 276-10': [37.56286, 127.08921],
 '서울 광진구 자양동 203-12': [37.5329735, 127.0727409],
 '서울 광진구 자양동 578-5': [37.5329735, 127.0727409],
 '서울 광진구 자양동 579': [37.5329735, 127.0727409],
 '서울 광진구 광장동 453-7': [37.54711, 127.10414],
 '서울 광진구 광장동 486-1': [37.54711, 127.10414],
 '서울 광진구 군자동 207-18': [37.5532345, 127.071936],
 '서울 광진구 능동 255-7': [37.55444, 127.08156],
 '서울특별시 광진구 능동로36길 172': [37.5545683, 127.084021],
 '서울특별시 광진구 능동로32길 121': [37.5521511, 127.0848497],
 '서울특별시 광진구 능동로 318': [37.55794535, 127.08018203349465],
 '서울 광진구 화양동 46-22': [37.54408, 127.06895],
 '서울 광진구 화양동 24-33': [37.54408, 127.06895],
 '서울 광진구 구의동 55-3': [37.54417, 127.09262],
 '서울 광진구 자양동 45-5': [37.5329735, 127.0727409],
 '서울특별시 광진구 뚝섬로 498-1'

----

In [203]:
geo_dict

{'서울 광진구 중곡동 482-1': [37.56286, 127.08921],
 '서울 광진구 구의동 632-3': [37.54417, 127.09262],
 '서울특별시 광진구 긴고랑로7길 76': [37.5640119, 127.0794286],
 '서울 광진구 중곡동 265-17': [37.56286, 127.08921],
 '서울 광진구 구의동 662-1': [37.54417, 127.09262],
 '서울 광진구 중곡동 128-13': [37.56286, 127.08921],
 '서울 광진구 중곡동 276-10': [37.56286, 127.08921],
 '서울 광진구 자양동 203-12': [37.5329735, 127.0727409],
 '서울 광진구 자양동 578-5': [37.5329735, 127.0727409],
 '서울 광진구 자양동 579': [37.5329735, 127.0727409],
 '서울 광진구 광장동 453-7': [37.54711, 127.10414],
 '서울 광진구 광장동 486-1': [37.54711, 127.10414],
 '서울 광진구 군자동 207-18': [37.5532345, 127.071936],
 '서울 광진구 능동 255-7': [37.55444, 127.08156],
 '서울특별시 광진구 능동로36길 172': [37.5545683, 127.084021],
 '서울특별시 광진구 능동로32길 121': [37.5521511, 127.0848497],
 '서울특별시 광진구 능동로 318': [37.55794535, 127.08018203349465],
 '서울 광진구 화양동 46-22': [37.54408, 127.06895],
 '서울 광진구 화양동 24-33': [37.54408, 127.06895],
 '서울 광진구 구의동 55-3': [37.54417, 127.09262],
 '서울 광진구 자양동 45-5': [37.5329735, 127.0727409],
 '서울특별시 광진구 뚝섬로 498-1'

In [204]:
#!pip install geocoder

In [205]:
geo_for_mapping = pd.DataFrame(geo_dict).T.reset_index()
geo_for_mapping.columns = ['단속장소','위도','경도']

In [206]:
geo_for_mapping

,단속장소,위도,경도
0,서울 광진구 중곡동 482-1,37.56286,127.08921
1,서울 광진구 구의동 632-3,37.54417,127.09262
2,서울특별시 광진구 긴고랑로7길 76,37.564012,127.079429
3,서울 광진구 중곡동 265-17,37.56286,127.08921
4,서울 광진구 구의동 662-1,37.54417,127.09262
...,...,...,...
12800,서울 광진구 구의동 광나루로56길 85,None,None
12801,서울 광진구 자양동 동일로18길 32,None,None
12802,서울특별시 광진구 아차산로55길 59,37.539749,127.086433
12803,서울특별시 광진구 뚝섬로52다길 17,37.530186,127.07988


In [207]:
df

,단속일시,과태료,단속지역,단속장소,위반내용,견인지시,단속구분
120065,2020-03-17,32000,자양동,서울 광진구 자양동 662-5,횡단보도,미견인,스마트앱(서울시)
120066,2020-03-17,40000,긴고랑로,서울 광진구 긴고랑로 40,횡단보도,미견인,생활불편앱(행안부)
120067,2020-03-17,40000,군자로,서울 광진구 군자로 164-1,보도,미견인,PDA
120068,2020-03-17,32000,중곡동,서울 광진구 중곡동 190-29,보도,미견인,스마트앱(서울시)
120069,2020-03-17,32000,아차산로65길,서울 광진구 아차산로65길 50,교통소통장애,미견인,PDA
...,...,...,...,...,...,...,...
206584,2023-03-31,32000,중곡동,서울 광진구 중곡동 192-30,보도,미견인,스마트앱(서울시)
206585,2023-03-31,64000,광장동,서울 광진구 광장동 320-15,특별구역,미견인,안전신문고(행안부)
206586,2023-03-31,32000,중곡동,서울 광진구 중곡동 669,보도,미견인,PDA
206587,2023-03-31,32000,중곡동,서울 광진구 중곡동 669,보도,미견인,PDA


In [208]:
df = df.merge(geo_for_mapping,on='단속장소')

In [209]:
df

,단속일시,과태료,단속지역,단속장소,위반내용,견인지시,단속구분,위도,경도
0,2020-03-17,32000,자양동,서울 광진구 자양동 662-5,횡단보도,미견인,스마트앱(서울시),37.532973,127.072741
1,2020-03-20,40000,자양동,서울 광진구 자양동 662-5,횡단보도,미견인,스마트앱(서울시),37.532973,127.072741
2,2020-03-23,32000,자양동,서울 광진구 자양동 662-5,횡단보도,미견인,스마트앱(서울시),37.532973,127.072741
3,2020-03-23,40000,자양동,서울 광진구 자양동 662-5,횡단보도,미견인,스마트앱(서울시),37.532973,127.072741
4,2020-03-29,40000,자양동,서울 광진구 자양동 662-5,횡단보도,미견인,스마트앱(서울시),37.532973,127.072741
...,...,...,...,...,...,...,...,...,...
86519,2023-03-31,32000,화양동,서울 광진구 화양동 9-86,교차로,미견인,PDA,37.54408,127.06895
86520,2023-03-31,32000,중곡동,서울 광진구 중곡동 150-57,보도,미견인,스마트앱(서울시),37.56286,127.08921
86521,2023-03-31,120000,구의동,서울 광진구 구의동 644-1,주차방법위반,미견인,PDA,37.54417,127.09262
86522,2023-03-31,32000,화양동,서울 광진구 화양동 110-67,교통소통장애,미견인,PDA,37.54408,127.06895


In [210]:
ill = pd.read_csv('인프라\\서울특별시 광진구_주정차단속현황_20230331.csv',encoding='cp949')
geos = pd.read_csv('FIXED_CCTV.csv')
ill = ill[ill['단속구분']=='고정형CCTV']
geos = geos[geos['단속지점명'].isin(ill['단속장소']) ][['위도','경도','단속지점명']]
geos.columns=['위도','경도','단속장소']
geos = ill.merge(geos)
df = pd.concat([df,geos])
df['단속일시'] = pd.to_datetime(df['단속일시'])
df = df[df['단속일시']>='2020-03-17']
# 2020-03-17 이후 데이터를 활용

In [211]:
df

,단속일시,과태료,단속지역,단속장소,위반내용,견인지시,단속구분,위도,경도
0,2020-03-17 00:00:00,32000,자양동,서울 광진구 자양동 662-5,횡단보도,미견인,스마트앱(서울시),37.532973,127.072741
1,2020-03-20 00:00:00,40000,자양동,서울 광진구 자양동 662-5,횡단보도,미견인,스마트앱(서울시),37.532973,127.072741
2,2020-03-23 00:00:00,32000,자양동,서울 광진구 자양동 662-5,횡단보도,미견인,스마트앱(서울시),37.532973,127.072741
3,2020-03-23 00:00:00,40000,자양동,서울 광진구 자양동 662-5,횡단보도,미견인,스마트앱(서울시),37.532973,127.072741
4,2020-03-29 00:00:00,40000,자양동,서울 광진구 자양동 662-5,횡단보도,미견인,스마트앱(서울시),37.532973,127.072741
...,...,...,...,...,...,...,...,...,...
55744,2023-03-23 16:30:00,32000,중곡동,하이프라자 광진군자점 인근,주정차금지(황색실선)구역,미견인,고정형CCTV,37.556815,127.080881
55745,2023-03-24 16:56:00,40000,중곡동,하이프라자 광진군자점 인근,주정차금지(황색실선)구역,미견인,고정형CCTV,37.556815,127.080881
55746,2023-03-29 07:14:00,40000,중곡동,하이프라자 광진군자점 인근,주정차금지(황색실선)구역,미견인,고정형CCTV,37.556815,127.080881
55747,2023-03-31 14:48:00,32000,중곡동,하이프라자 광진군자점 인근,주정차금지(황색실선)구역,미견인,고정형CCTV,37.556815,127.080881


In [212]:
df.to_csv('광진구_위경도_단속.csv',index=False)
# 저장